# Factorization Machine

之前的　Linear Regression 的形式是形如　$\hat{y}= w^T x$，Factorization Machine 在　Linear Regression　的基础之上添加了所谓的交叉项，即　$w_{ij}x_ix_j$, 即　$\hat{y} = w_1x_1 + \ldots + w_nx_n + \sum_{i=1}^{n-1} \sum_{j=i+1}^{n} w_{ij}x_ix_j$，由于有些交叉项在实际中并不存在，所以使用矩阵分解的办法，用一个　$n \times k$　的矩阵，使用矩阵相乘的办法得到一个 $n \times n$　的矩阵。

最后的交叉项可以写成　$\sum_{i=1}^{n-1} \sum_{j=i+1}^{n} w_{ij}x_ix_j = \frac{1}{2}(\sum_{i=1}^n \sum_{j = 1}^n w_{ij}x_ix_j - \sum_i^n w_{ii}x_ix_i) = \frac{1}{2} \sum_{f}^k[(\sum_i^n v_{if}x_i)^2 - \sum_i^nv_{if}^2 x_i^2]$，降低计算复杂度。

其中 $w = v \times v$

计算　Loss 采用的函数仍是　MSE，即　$loss = (\hat{y} - y)^2$

Gradient 的计算及更新：

$\begin{align*}
w_i & = w_i - \eta \cdot (\hat{y} - y) \cdot x_i \\
v_{ij} & = v_{ij} - \eta \cdot (\hat{y} - y) \cdot (x_i \sum_k^n v_{kj}x_k - x_i^2v_{ij})
\end{align*}$